# Working with Delta Tables using MLTable

#### This notebook includes an example on how to consume a delta table in AML, and register that table as a data asset using MLTable.
#### For additional information, refer to this [documentation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-mltable?view=azureml-api-2)

**Note:** Please ensure the version of mltable is >=1.3.0

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import AzureDataLakeGen2Datastore, ServicePrincipalConfiguration, AzureBlobDatastore, AccountKeyConfiguration
from azure.ai.ml.operations import DatastoreOperations
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azureml.fsspec import AzureMachineLearningFileSystem
import mltable

from importlib.metadata import version

In [ ]:
version('mltable')

In [ ]:
#Enter details of your AzureML workspace

subscription_id = ''
resource_group = ''
workspace = ''
datastorename = ''
path_on_datastore = ''

# long-form Datastore uri format:
# uri = f'azureml://subscriptions/{subscription}/resourcegroups/{resource_group}/workspaces/{workspace}/datastores/{datastore_name}/paths/{path_on_datastore}'
path = f'azureml://subscriptions/{subscription_id}/resourcegroups/{resource_group}/workspaces/{workspace}/datastores/{datastorename}/paths/{path_on_datastore}'

delta = mltable.from_delta_lake(delta_table_uri=path, version_as_of=0)



In [ ]:
delta.to_pandas_dataframe()

In [ ]:
print(delta)

In [ ]:
delta.save('mltable')
# This saves the mltable file locally, in a newly created folder called "mltable". This file could, for example, be moved to the datalake next to the data

In [ ]:
loaded_delta = mltable.load('mltable')

In [ ]:
loaded_delta.to_pandas_dataframe()

In [ ]:

ml_client = MLClient(credential=DefaultAzureCredential(),
                     subscription_id=subscription_id, 
                     resource_group_name=resource_group,
                     workspace_name=workspace)

In [ ]:
# This step is optional, but could be a good practice to keep this configuration file in the same place as the data
# Alternatively, this file can remain local.
fs = AzureMachineLearningFileSystem(path) # create a fsspec filesystem for the datastore
fs.upload(lpath='mltable/MLTable', rpath=path_on_datastore) # upload the mltable file to the datastore

In [ ]:
# After the mltable configuration file is created, a data asset can be created, effectively pointing to the config file
my_data_asset = Data(
    path=path, # this is the path to the mltable file in the datalake. Alternatively, this can be a local path
    type=AssetTypes.MLTABLE,
    description="delta table data asset",
    name="delta_data_asset",
    version='1'
)

In [ ]:
ml_client.data.create_or_update(my_data_asset)

In [ ]:
# This cell ilustrates how to load a data asset after it has been registered

VERSION="1"
data_asset = ml_client.data.get(name="delta_data_asset", version=VERSION)

# create a table
tbl = mltable.load(f"azureml:/{data_asset.id}")
tbl.show(5)

# load into pandas
df = tbl.to_pandas_dataframe()